In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [37]:
train_generator = train_datagen.flow_from_directory(
        '../data/archive/train/mouth-cropped',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

validation_generator = train_datagen.flow_from_directory(
        '../data/archive/val/mouth-cropped',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 856 images belonging to 2 classes.
Found 266 images belonging to 2 classes.


In [5]:
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False, weights='imagenet', input_shape=(80,80,3)
)

In [28]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False, weights='imagenet', input_shape=(80,80,3)
)

In [29]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [38]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(80, 80, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.5),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [39]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [45]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 78, 78, 16)        448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 39, 39, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 37, 37, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 18, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 8, 8, 64)        

In [41]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=50,
      verbose=1,
      callbacks=early_stop)

Epoch 1/50


54/54 [==============================] - 2s 26ms/step - loss: 0.4218 - accuracy: 0.8084 - val_loss: 0.2169 - val_accuracy: 0.9361
Epoch 2/50
54/54 [==============================] - 1s 17ms/step - loss: 0.2457 - accuracy: 0.9124 - val_loss: 0.6143 - val_accuracy: 0.7218
Epoch 3/50
54/54 [==============================] - 1s 20ms/step - loss: 0.2018 - accuracy: 0.9369 - val_loss: 0.2558 - val_accuracy: 0.8872
Epoch 4/50
54/54 [==============================] - 1s 25ms/step - loss: 0.1604 - accuracy: 0.9533 - val_loss: 0.1618 - val_accuracy: 0.9549
Epoch 5/50
54/54 [==============================] - 1s 25ms/step - loss: 0.1517 - accuracy: 0.9544 - val_loss: 0.1561 - val_accuracy: 0.9586
Epoch 6/50
54/54 [==============================] - 1s 27ms/step - loss: 0.1284 - accuracy: 0.9614 - val_loss: 0.1849 - val_accuracy: 0.9361
Epoch 7/50
54/54 [==============================] - 1s 19ms/step - loss: 0.1317 - accuracy: 0.9614 - val_loss: 0.2284 - val_accuracy: 0.9173
Epoch 8/50
54/54 [======

In [42]:
test_generator = train_datagen.flow_from_directory(
        '../data/test/yourfolderforcroppedmouths',  # This is the source directory for training images
        target_size=(80, 80),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 46 images belonging to 2 classes.


In [44]:
model.evaluate(test_generator)

3/3 [==============================] - 0s 17ms/step - loss: 0.2241 - accuracy: 0.9130


[0.22405843436717987, 0.9130434989929199]

In [46]:
model.save("base_cropped_yawning_model.h5")